### 포리움을 활용한 지도 만들기

- 지난 시간에 우리는 판다스 라이브러리와 소상공인 데이터를 이용하여 원하는 데이터를 추출하고 상호명에 차이가 있는 프렌차이즈 업체의 이름을 브랜드명으로 통일시켜주는 작업을 수행했다.
- 그리고 이 수행결과를 파일로 저장하였다.
- 이번시간에는 저번 시간에 만들었던 파일과 포리움 라이브러리를 이용하여 강원도 파리바게트, 배스킨라빈스 지도를 만드는 활동을 수행해보자

### 우리 집 주소의 위치를 어떻게 숫자로 표현할까?
- 일단 지도를 만들기 전에 지도의 시작점을 생각해보자.
- 지도의 시작점은 내가 가장 익숙한 우리 집 근처가 좋을 것이다.
- 그렇다면 우리 집 주소를 컴퓨터가 이해할 수 있는 숫자로 바꾸는 작업이 필요하다.
- 지도를 만들기 위해 필요한 숫자는 위도와 경도이다.
- 인터넷 검색을 통해 우리 집 위치를 위도와 경도로 구할 수 있지만
- 오픈 API를 통한 반응형 위도, 경도 출력기를 만들어보자.

### 오픈 API

- 오픈 API는 다운로드 받을 수 있는 데이터 파일과는 달리 어떤 '서비스'를 제공해주는 환경이다.
- 오픈 API를 사용하기 위해서는 '키'를 받아야 하며 키와 함께 내가 알고 싶은 정보를 '입력'해 주어야 한다.
- 키와 정보 요청을 입력해주면 결과값이 출력된다.
- 그런데 그 결과값은 깔끔한 형태가 아니기에 이를 깔끔하게  처리해주는 작업이 필요하다.
- 키보드로 우리 집 주소(도로명)을 입력하면 위도와 경도를 출력하고 그 값을 저장해주는 프로그램을 만들어보자.
- VWorld에서 제공하는 오픈 API를 활용하면 이를 만들 수 있다.
- https://www.vworld.kr/dev/v4dv_geocoderguide2_s001.do

In [ ]:
import requests, bs4
import json
from urllib.parse import unquote, quote_plus, urlencode

# 집 주소 입력하면 그대로 출력해주는 프로그램

address = input("우리집 주소를 도로명으로 입력해주세요 : ")
print(address)

# API에 필요한 주소, 키, 키를 수정한 값

url = 'http://api.vworld.kr/req/address?service=address&request=GetCoord' #정보를 요청하는 오픈 API주소
API_KEY = '60821C27-D454-3290-A84E-07A4BED720B6' #오픈 API 키
MY_API_KEY = unquote(API_KEY) #사전을 유니코드로 통일시켜주기 

# 요청사항을 담은 question (포트폴리오!!!)

question = '&' + urlencode(
    { #인터넷 주소 요청시 사전 통일
            quote_plus('key') : MY_API_,  
            quote_plus('address') : ,
            quote_plus('type') : 'road',
            quote_plus('format') : 'json',
            quote_plus('crs') : 'epsg:4326'
    }
)

print(url + question)

response = requests.get(url + question).text
print(response)

geo_info = .loads(response) #포트폴리오!!! json 포맷 형식으로 읽어와 저장하기
lat = geo_info['response']['result']['']['y'] #포트폴리오!!!
lon = geo_info['response']['result']['point'][''] #포트폴리오!!!

print(lat) #위도값
print(lon) #경도값
lat = float(lat) #실수값으로 만들어줌 
lon = float(lon) #실수값으로 만들어줌

### Folium 라이브러리를 이용한 시각화

- 이제 우리집의 위도와 경도를 구했으니 시각화 작업을 진행하면 된다.
- 위도와 경도가 있는 데이터라면 이를 기반으로 지도를 만들어줄 수 있다.
- 지도 위에 위치를 나타내는 마커를 찍어주고 마커에 간단한 정보를 넣어주도록 하자!

In [ ]:
import folium # 포리움 라이브러리 

m = folium.Map([lat,lon]) #지도의 시작값을 지정하고 만들어줌
folium.Marker([lat,lon], popup='우리집', tooltip="짱").add_to(m) #팝업은? () 툴팁은? ()
m #지도를 보여줌

In [ ]:
#판다스 데이터 프레임 형태로 배스킨라빈스와 파리바게트 정보를 불러옴
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/CarlosQuperman/sdhs202121/main/%EC%B6%98%EC%B2%9C(%EB%B0%B0%EC%8A%A4%ED%82%A8_%ED%8C%8C%EB%A6%AC).csv")
df = df.fillna(0) #비어있는 값들을 0으로 채워줌  
df.head()

In [11]:
# 불러온 데이터의 크기 확인

df.shape

(170, 11)

In [ ]:
# 불러온 데이터의 이름 및 종류(데이터형) 확인

df.info()

### 파리바게트 배스킨라빈스 지도 만들어주기

- 이제 데이터도 불러왔으니 준비는 다 완료
- 어떻게 지도에 마커를 찍어줄까?
- 처음부터 끝까지 df를 돌면서
- 위도와 경도가 비어있지 않다면 마커를 찍어주는데
- 툴팁과 팝업에 상호명과 도로명주소를 표시해보도록 하자

In [ ]:
m = folium.Map([lat,lon],zoom_start=20) #지도의 시작값과 확대 정도를 지정하고 만들어줌

#판다스 데이터 프레임은 row in df로 직접 돌 수가 없음 그래서 위치를 의미하는 인덱스를 사용 
for i in df. : #(포트폴리오!!!) 위치번호(index)로 처음부터 끝까지 돌면서 
  if df.loc[i,'위도']!='0' and df.loc[i,'경도']!='0':
    # (포트폴리오!!!) 우리가 필요한 정보는 위도, 경도, 상호명, 주소 
    lat2 = df.loc[,'']
    lon2 = df.loc[i,'']
    title = df.loc[i,'']
    address = df.loc[i,'']

    icon_color = '' #색지정 (포트폴리오!!!) (파란색으로)

    folium.Marker([lat2,lon2], popup=address, tooltip=title,icon=folium.Icon(color=icon_color)).add_to(m)

m


### 위 지도를 좀 더 구체화 해보자

- 파리바게뜨와 배스킨라빈스가 전부 파란색으로 되어 있어서 잘 분류가 되어 있지 않다.
- 배스킨라빈스의 경우는 핑크색으로 표시해보자!

In [ ]:
m = folium.Map([lat,lon],zoom_start=20) #지도의 시작값과 확대 정도를 지정하고 만들어줌

#판다스 데이터 프레임은 row in df로 직접 돌 수가 없음 그래서 위치를 의미하는 인덱스를 사용 
for i in df.index : #처음부터 끝까지 돌면서
  if df.loc[i,'위도']!='0' and df.loc[i,'경도']!='0':
    # 우리가 필요한 정보는 위도, 경도, 상호명, 주소 
    lat2 = df.loc[i,'위도']
    lon2 = df.loc[i,'경도']
    title = df.loc[i,'상호명']
    address = df.loc[i,'도로명주소']

    icon_color = 'blue' #색지정

    if df.loc[,'']=='배스킨라빈스': #(포트폴리오!!!) i번째에 있는 브랜드명이 배스킨라빈스인지 질문한다.
       icon_color = 'pink'

    folium.Marker([lat2,lon2], popup=address, tooltip=title,icon=folium.Icon(color=icon_color)).add_to(m)

m


### 지역별로 파리바게뜨와 배스킨라빈스의 분포를 쉽게 볼 수 없을까?

- 마커가 뭉쳐있는 곳에서는 잘 구별이 되지 않아 확인하기가 어렵다.
- 지역별로 마커가 몇 개인지 확인하고 확대했을 때 자세한 마커를 보여주면 더 효과적일 것이다.
- 이때에는 포리움의 마커클러스터를 활용하면 좋다!

In [ ]:
from folium.plugins import MarkerCluster # 1 마커 클러스터 라이브러리 불러오기

m = folium.Map([lat,lon],zoom_start=20) 
marker_cluster = MarkerCluster().add_to() #2 (포트폴리오!!!) 마커 클러스터에 위에서 만든 지도 m 추가

#판다스 데이터 프레임은 row in df로 직접 돌 수가 없음 그래서 위치를 의미하는 인덱스를 사용 
for i in df.index : #처음부터 끝까지 돌면서
  if df.loc[i,'위도']!='0' and df.loc[i,'경도']!='0':
    # 우리가 필요한 정보는 위도, 경도, 상호명, 주소 
    lat2 = df.loc[i,'위도']
    lon2 = df.loc[i,'경도']
    title = df.loc[i,'상호명']
    address = df.loc[i,'도로명주소']

    icon_color = 'blue' #색지정

    if df.loc[i,'브랜드명']=='배스킨라빈스':
       icon_color = 'pink'

    folium.Marker([lat2,lon2], popup=address, tooltip=title,icon=folium.Icon(color=icon_color)).add_to() #3 (포트폴리오!!!) 기존의 m을 마커 클러스터로 바꿔준다

m


In [ ]:
# 컴퓨터로 하는 사람만!!

from google.colab import drive
drive.mount('/content/gdrive')

# 자기 자신의 구글 드라이브에 접근할 수 있는 권한 (사용 이유 : 데이터 파일을 읽어오고 프로그래밍과 시각화 진행 후 결과값을 파일로 출력하기 위해서)

In [ ]:
m.save('') #위에서 만들어진 지도를 파일로 저장

### 선택 과제

- 이번 차시와 저번 차시에 학습한 내용을 활용한다면
- 여러분들은 위도와 경도가 있는 어떤 CSV 데이터가 있을 때든 
- 원하는 형태로 추출하고 지도로 만들 수 있다
- 오늘 배운 내용을 조금씩 바꿔보거나
- 내가 관심 있는 데이터를 찾아 이를 전처리, 추출, 시각화해보자

In [ ]:
# 예시 데이터

#df_cctv = pd.read_csv("https://raw.githubusercontent.com/CarlosQuperman/sdhs202121/main/%EA%B0%95%EC%9B%90%EB%8F%84CCTV.csv")
#df_cctv = df_cctv.fillna(0) #비어있는 값들을 0으로 채워줌 

